# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [13]:
def priest(sex, age, respiRt, OxySatu, heartRt, systolicbp, temp, alrt, inOxy, perfSts):

    """

    (String, float, float, float, float, float, float, String, String, String) -> float)

    The priest method is used to determine the percentage of the risk
    of the clinical Severity Score 'PRIEST' COVID-19.

    Priest function accepts sex, alrt, inOxy and perfSts in String and remaining all other arguments are in float.

    >>> priest("male",65,22,0.96,41,115,38.1,"ALERT", "AIR", "unrestricted normal activity")
    0.18

    >>> priest("Female",40,22,0.98,89,230,38.1,"alert","AIR","unrestricted normal activity")
    0.15

    >>> priest("FEMALE",18,22,0.96,50,115,38.1,"ALERT","AIR","limited strenuous activity, can do light activity")
    0.09

    """


    prscore = 0

    if age >= 50 and age<=65:
        prscore = prscore + 2
    elif age >= 66 and age<=80:
        prscore = prscore + 3
    elif age > 80:
        prscore = prscore + 4


    if alrt.lower() == "confused or not alert":
        prscore = prscore +  3

    if perfSts.lower() == "limited self-care":
        prscore = prscore +  3
    elif perfSts.lower() == "limited activity, can self-care":
        prscore = prscore +  2
    if perfSts.lower() == "limited strenuous activity, can do light activity":
        prscore = prscore +  1
    elif perfSts.lower() == "bed/chair bound, no self-care":
        prscore = prscore +  4

    if inOxy.lower() == "supplemental oxygen":
        prscore = prscore +  2

    if respiRt < 9:
        prscore = prscore +  3
    if respiRt > 24:
        prscore = prscore +  3
    elif respiRt >= 21 and respiRt <= 24:
        prscore = prscore +  2
    elif respiRt >= 9 and respiRt <= 11:
        prscore = prscore +  1

    if OxySatu < 0.92:
        prscore = prscore +  3
    elif OxySatu >= 0.94 and OxySatu <= 0.95:
        prscore = prscore +  1
    elif OxySatu >= 0.92 and OxySatu <= 0.93:
        prscore = prscore +  2

    if heartRt > 130:
        prscore = prscore +  3
    elif heartRt >= 111 and heartRt <= 130:
        prscore = prscore +  2
    elif heartRt >= 91 and heartRt <= 110:
        prscore = prscore +  1
    elif heartRt < 41:
        prscore = prscore +  3
    elif heartRt >= 41 and heartRt <= 50:
        prscore = prscore +  1


    if systolicbp > 219:
        prscore = prscore +  3
    elif systolicbp >= 101 and systolicbp <= 110:
        prscore = prscore +  1
    elif systolicbp >= 91 and systolicbp <= 100:
        prscore = prscore +  2
    elif systolicbp < 91:
        prscore = prscore +  3

    if temp > 39.0:
        prscore = prscore +  2
    elif temp >= 38.1 and temp <= 39.0:
        prscore = prscore +  1
    if temp < 35.1:
        prscore = prscore +  3
    elif temp >= 35.1 and temp <= 36.0:
        prscore = prscore +  1

    if sex.lower()== "male":
        prscore = prscore + 1


    scale = [0.01, 0.01, 0.02, 0.02, 0.03, 0.09, 0.15, 0.18, 0.22, 0.26, 0.29, 0.34, 0.38, 0.46, 0.47, 0.49, 0.55, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,0.99]
    if prscore < 26:
        return scale[prscore]



In [11]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("male",65,22,0.96,41,115,38.1,"ALERT", "AIR", "unrestricted normal activity")
Expecting:
    0.18
**********************************************************************
File "__main__", line 12, in NoName
Failed example:
    priest("male",65,22,0.96,41,115,38.1,"ALERT", "AIR", "unrestricted normal activity")
Exception raised:
    Traceback (most recent call last):
      File "/usr/lib/python3.10/doctest.py", line 1350, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest NoName[0]>", line 1, in <module>
        priest("male",65,22,0.96,41,115,38.1,"ALERT", "AIR", "unrestricted normal activity")
      File "<ipython-input-10-f4e92c9c01b0>", line 95, in priest
        if sex.lower()== male:
    NameError: name 'male' is not defined
Trying:
    priest("Female",40,22,0.98,89,230,38.1,"alert","AIR","unrestricted normal activity")
Expecting:
    0.15
******************************************************************

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [12]:
import requests
import json


def find_hospital(age, sex, rsk):
    """

    (integer,string,float)->string


    Function Find_Hospital Find a hospital using the age, sex, and risk percentage.
    It generates hospital name in String after expecting age, risk percentage in float, and sex in String.

    The find_hospital command Passing age, sex, and rsk as query parameters to the
    url using the requests module will help you find the hospital on the API.


    >>> find_hospital(40, "female", 0.15)
    'Select Specialty Hospital - Northeast Atlanta'

    >>> find_hospital(40, "MALE", 0.22)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(16, "Male", 0.29)
    'Childrens Healthcare of Atlanta at Scottish Rite'

    """
    value={"age": age, "sex": sex.lower(), "risk_pct": rsk}
    final = requests.get("https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/", params=value)
    final2 = final.json()
    return final2['hospital']

In [14]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, "female", 0.15)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(40, "MALE", 0.22)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(16, "Male", 0.29)
Expecting:
    'Childrens Healthcare of Atlanta at Scottish Rite'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [15]:
def get_address(name):
    """
    (string)->string

    get_address function find the address of hospital based on hospital name from the url provided

    get_address function expects hospital name as String and return address in String

    get_address function find address by use API, where it has hospital name json data in it.

    >>> get_address("Childrens Healthcare of Atlanta at Scottish Rite")
    '1001 JOHNSON FERRY RD'

    >>> get_address("Select Specialty Hospital - Northeast Atlanta")
    '1821 CLIFTON ROAD NE'

    >>> get_address("Wesley Woods Geriatric Hospital")
    '1821 CLIFTON ROAD, NE'

    """

    urldata = req.get("https://hds5210-data.s3.amazonaws.com/ga_hospitals.json")

    name=name.upper()

    finall = urldata.json()

    if name in finall:

        final3 = finall[name]['ADDRESS']

        return final3

    return "Not found"

## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [16]:
import csv

def process_people(url):
    """

    (string)->dictionary

    All patient information, including risk, hospital, and
    hospital address, will be produced in a dictionary by this process_people method.


    process_people accepts a string input parameter for the url and outputs a dictionary with patient information.

    """
    lastOut = {}
    res = req.get(url)
    first = res.text.split("\n")
    second = csv.reader(first, delimiter='|')
    next(second)
    for k in second:
        if not any(field != '' for field in k):
            break

        patient, sex, age, respiRt, OxySatu, heartRt, systolicbp, temp, alrt, inOxy, perfSts = k

        rsk = priest(sex, float(age), float(respiRt), float(OxySatu), float(heartRt), float(systolicbp), float(temp), alrt, inOxy, perfSts)

        name = find_hospital(int(age), sex, rsk)

        erow = [sex, float(age), float(respiRt), float(OxySatu), float(heartRt), float(systolicbp), float(temp), alrt, inOxy, perfSts, rsk, name, get_address(name)]

        lastOut[patient] = erow


    return lastOut

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [17]:
import requests as req
import json
"""
It compares the json data from the link with the calculated data using above functions

It returns "Data is equal" when they are equal and not equal if they are not
"""

headers = {'Accept' : 'application/json'}
outdataValues = requests.get("https://hds5210-data.s3.amazonaws.com/people_results.json", headers=headers)
urldict = outdataValues.json()
urldictstring = json.dumps(urldict)
urldictstringfinal = json.loads(urldictstring)

processdict = process_people("https://hds5210-data.s3.amazonaws.com/people.psv")
processdictstring = json.dumps(processdict)
processdictstringfinal = json.loads(processdictstring)

if urldictstringfinal == processdictstringfinal:
    print("EQUAL")
else:
    print("NOT EQUAL")

EQUAL


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---